In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt

In [3]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle
import numpy as np
import string
import sys

In [4]:
import sys
sys.path.append(".\Income_tax_function")
import IRPP_from_scratch as ir

In [5]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Data/hdf/edp_2015_final.h5'
Store = pd.HDFStore(store_path)

In [6]:
def load_fisc_i_by_year(year = 2014):
    df_fisci = pd.read_hdf(store_path, 'FISC_INDIVIDU_{}'.format(year))
    return df_fisci
def load_fisc_r_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_REVENU_{}'.format(year))
    return df_fiscr
def load_fisc_l_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_LOGEMENT_{}'.format(year))
    return df_fiscr

def load_df_fiscrevdet_by_year(year = 2014):
    df_fiscrevdet = pd.read_hdf(store_path, 'FISC_REVDET_{}'.format(year))
    return df_fiscrevdet
    

In [7]:
df_fisci = load_fisc_i_by_year(2014)

Déclarants des couples mariés et pacés en fonction du sexe.

In [8]:
df_fisci["CIDECI"] = df_fisci.CIDECI.fillna("Error")
df_fisci.loc[df_fisci.SEXE=="C", "SEXE"]= 1
df_fisci["SEXE"] = df_fisci.SEXE.astype('float')

In [9]:
df_fisci.SEXE.value_counts()

2    2765880
1    2695414
dtype: int64

In [10]:
%%time
grpby = df_fisci[df_fisci.TYPE_FISC.isin(["1","2"])].groupby("ID_FISC_LOG_DIFF").CIDECI.sum()

Wall time: 2min 2s


In [11]:
married_couples_id = grpby[grpby =="MM"].index
df_fisci_married = df_fisci[df_fisci.ID_FISC_LOG_DIFF.isin(married_couples_id)]
#df_fisci_married.loc[df_fisci_married.SEXE=="C", "SEXE"]=np.NaN
df_fisci_married.loc[:,"SEXE"] = df_fisci_married.SEXE.astype('float')
((df_fisci_married.TYPE_FISC == "2") & (df_fisci_married.SEXE == 1)).value_counts(normalize = True)
((df_fisci_married.TYPE_FISC == "2") & (df_fisci_married.SEXE == 2)).value_counts(normalize = True)

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\pandas\core\indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


False    0.698427
True     0.301573
dtype: float64

In [12]:
df_fisci_married.SEXE.value_counts()

1    1146472
2    1140934
dtype: int64

In [13]:
((df_fisci_married.TYPE_FISC == "2") & (df_fisci_married.SEXE == 1)).value_counts(normalize = True)

False    0.997646
True     0.002354
dtype: float64

In [14]:
crosstab = pd.crosstab(df_fisci_married.SEXE, df_fisci_married.TYPE_FISC, margins = True).iloc[:,0:2]
table = pd.concat([crosstab,crosstab/crosstab.loc["All"]] , axis = 1); table

TYPE_FISC,1,2,1,2
SEXE,,,,
1.0,1065555,8315,0.992373,0.007744
2.0,8189,1065402,0.007627,0.992246
All,1073744,1073728,1.000000,1.000000


## Plus riche prennant les enfants

In [15]:
path = (u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Programme/optimisation_concubains/" +
                    "/Re_restart_from_scratch/pickle/final_select_rev_{}.p".format(2014))
import pickle
select_id_fisc_log = pickle.load(
            open(path, 'rb'))

In [16]:
%%time
df_select = df_fisci[df_fisci.ID_FISC_LOG_DIFF.isin(select_id_fisc_log)]

Wall time: 2.06 s


In [17]:
a_charge_type_fisc_label =  [str(x) for x in range(3,10)]+list(string.ascii_uppercase)[:12]

In [18]:
df_pers_a_charge_by_foy = df_select[df_select.TYPE_FISC.isin(
       a_charge_type_fisc_label
            )].groupby("ID_FISC_FOY_DIFF").count()

df_pers_a_charge_nb_by_foy = pd.DataFrame(df_pers_a_charge_by_foy.AN_FISC)
df_pers_a_charge_nb_by_foy = df_pers_a_charge_nb_by_foy.rename(columns={'AN_FISC':'Nb_pers_a_charge_in_fiscal_h'}).reset_index()
df_selec_2 = pd.merge(df_select, df_pers_a_charge_nb_by_foy, on= ["ID_FISC_FOY_DIFF"], how = "left" )

In [19]:
assert df_select.shape[0] == df_selec_2.shape[0]

In [20]:

df_pers_a_charge_by_log = df_select[df_select.TYPE_FISC.isin(
       a_charge_type_fisc_label
                        )].groupby("ID_FISC_LOG_DIFF").count()

df_pers_a_charge_nb_by_log = pd.DataFrame(df_pers_a_charge_by_log.AN_FISC)
df_pers_a_charge_nb_by_log = df_pers_a_charge_nb_by_log.rename(columns={'AN_FISC':'Nb_pers_a_charge_in_household'}).reset_index()
df_selec_3 = pd.merge(df_selec_2.reset_index(), df_pers_a_charge_nb_by_log, on= ["ID_FISC_LOG_DIFF"], how = 'left' )

In [21]:
assert df_select.shape[0] == df_selec_3.shape[0]

In [ ]:
df_selec_3.sort('ID_FISC_LOG_DIFF')[
    ["Nb_pers_a_charge_in_fiscal_h", "Nb_pers_a_charge_in_household",
     "ID_DIFF","AN_FISC","ID_FISC_FOY_DIFF","ID_FISC_LOG_DIFF","ORDREFIP","TYPE_FISC","SEXE","JNAIS","MNAIS","ANAIS",
     ]
                                    ].head()

In [23]:
df_fiscrevdet = load_df_fiscrevdet_by_year(2014)

In [24]:
df_fiscrevdet["Revenus"] = df_fiscrevdet[["YSALI","YCHOI","YRSTI","YALRI","YRAGI","YBICI","YBNCI"]].sum(axis = 1)

In [25]:
merge = pd.merge(df_selec_3,df_fiscrevdet[["Revenus", "ID_FISC_FOY_DIFF"]] , on = "ID_FISC_FOY_DIFF", how = "left")

In [ ]:
merge.head()

In [27]:
merge['Revenus_max_by_log'] = merge.groupby('ID_FISC_LOG_DIFF').Revenus.transform(max)

In [28]:
merge["Is_rev_max"] = merge.groupby('ID_FISC_LOG_DIFF').Revenus.transform(max) == merge.Revenus

In [29]:
merge.Nb_pers_a_charge_in_fiscal_h.fillna(0, inplace = True)

In [30]:
merge['Has_all_child'] = merge.Nb_pers_a_charge_in_fiscal_h == merge.Nb_pers_a_charge_in_household

In [31]:
merge['Has_most_child'] = (
    merge.groupby('ID_FISC_FOY_DIFF').Nb_pers_a_charge_in_fiscal_h.transform(max)/merge.Nb_pers_a_charge_in_household)  >= 0.5

In [ ]:
merge.head()

### Have All children

In [33]:
crosstab = pd.crosstab(merge.groupby("ID_FISC_FOY_DIFF").first().Is_rev_max, merge.groupby("ID_FISC_FOY_DIFF").first().Has_all_child, margins=True)

In [34]:
crosstab

Has_all_child,False,True,All
Is_rev_max,,,
False,135939,26955,162894
True,71929,96935,168864
All,207868,123890,331758


In [35]:
crosstab/crosstab.loc["All","All"]

Has_all_child,False,True,All
Is_rev_max,,,
False,0.409753,0.081249,0.491002
True,0.216812,0.292186,0.508998
All,0.626565,0.373435,1.000000


### Have Most children

In [36]:
crosstab_2 = pd.crosstab(merge.groupby("ID_FISC_FOY_DIFF").first().Is_rev_max, merge.groupby("ID_FISC_FOY_DIFF").first().Has_most_child, margins=True)

In [37]:
crosstab_2

Has_most_child,False,True,All
Is_rev_max,,,
False,101401,61493,162894
True,34224,134640,168864
All,135625,196133,331758


In [38]:
crosstab_2/crosstab_2.loc[:,"All"]

Has_most_child,False,True,All
Is_rev_max,,,
False,0.622497,0.364157,0.491002
True,0.210100,0.797328,0.508998
All,0.832597,1.161485,1.000000


In [39]:
grpby_first = merge.groupby("ID_FISC_FOY_DIFF").first()

In [40]:
grpby_first[grpby_first.Has_all_child == True].Is_rev_max.value_counts(normalize = True)

True     0.782428
False    0.217572
dtype: float64

In [41]:
(merge.groupby("ID_FISC_FOY_DIFF").first().Has_most_child & merge.groupby("ID_FISC_FOY_DIFF").first().Is_rev_max).value_counts(normalize = True)

False    0.594162
True     0.405838
dtype: float64

In [42]:
(merge.groupby("ID_FISC_FOY_DIFF").first().Has_all_child & merge.groupby("ID_FISC_FOY_DIFF").first().Is_rev_max).value_counts(normalize = True)

False    0.707814
True     0.292186
dtype: float64

### Pere prennant les enfants

Question: Est-ce que les pères prennent plus enfants parce qu'ils sont plus riche ou est-ce parcequ'ils sont "mâles"  ?

In [43]:
df_sexe = merge[merge.TYPE_FISC.isin(["1"])]

In [44]:
df_sexe["SEXE"] = df_sexe.SEXE.astype("float")

C:\Users\IMPTEMP_A_PACIFIC\Desktop\WinPython-64bit-2.7.10.2\WinPython-64bit-2.7.10.2\python-2.7.10.amd64\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [45]:
df_sexe[df_sexe.Has_most_child].SEXE.value_counts(normalize = True)

1    0.558206
2    0.441794
dtype: float64

In [46]:
df_sexe[df_sexe.Is_rev_max].SEXE.value_counts(normalize = True)

1    0.70653
2    0.29347
dtype: float64

In [47]:
df_sexe = df_sexe[df_sexe.Nb_pers_a_charge_in_household == 1]

In [48]:
crosstab = pd.crosstab(df_sexe.Is_rev_max, df_sexe.Has_most_child, margins=True)

In [49]:
crosstab

Has_most_child,False,True,All
Is_rev_max,,,
False,43485,16157,59642
True,16833,44175,61008
All,60318,60332,120650


In [50]:
df_sexe[df_sexe.Is_rev_max].SEXE.value_counts(normalize = True)

1    0.670863
2    0.329137
dtype: float64

In [51]:
crosstab = pd.crosstab(df_sexe.Is_rev_max, df_sexe.SEXE, margins=True)

In [52]:
crosstab/crosstab.loc["All"]

SEXE,1.0,2.0,All
Is_rev_max,,,
False,0.319477,0.668143,0.494339
True,0.680523,0.331857,0.505661
All,1.000000,1.000000,1.000000


In [53]:
1-pd.DataFrame(pd.pivot_table(df_sexe, values= ["SEXE"], columns=["Is_rev_max", "Has_most_child"], aggfunc='mean', margins = True)-1)

0
     Is_rev_max Has_most_child          
SEXE False      False           0.357503
                True            0.227022
     True       False           0.759698
                True            0.637012
     All                        0.498483

In [60]:
biologic_path = (u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Programme/optimisation_concubains/" +
                "/Re_restart_from_scratch/pickle/biologic/biologic_households{}.p".format(year))
biologic = pickle.load( 
        open(biologic_path, 'rb'))
df_sexe_bio = df_sexe.reset_index()
df_sexe_bio = df_sexe_bio[df_sexe_bio.ID_FISC_LOG_DIFF.isin(biologic)]

In [61]:
optimize_path = (u"C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2015/Programme/optimisation_concubains/" +
                    "/Re_restart_from_scratch/pickle/optimize/Do_not_optimize_rev_{}.p".format(2014))
biologic = pickle.load(open(optimize_path, 'rb'))
df_sexe_bio["Do_not_optimize"]= df_sexe_bio.ID_FISC_LOG_DIFF.isin(biologic)

In [62]:
1-pd.DataFrame(pd.pivot_table(df_sexe_bio, values= ["SEXE"], columns=["Is_rev_max", "Has_most_child"], aggfunc='mean', margins = True)-1)

0
     Is_rev_max Has_most_child          
SEXE False      False           0.327512
                True            0.268644
     True       False           0.724516
                True            0.670003
     All                        0.499964

In [63]:
1-pd.DataFrame(pd.pivot_table(df_sexe_bio[df_sexe_bio.Do_not_optimize]
                              , values= ["SEXE"], columns=["Is_rev_max", "Has_most_child"], aggfunc='mean', margins = True)-1)

0
     Is_rev_max Has_most_child          
SEXE False      False           0.452263
                True            0.259712
     True       False           0.740288
                True            0.547588
     All                        0.499960

In [64]:
print df_sexe_bio[df_sexe_bio.Do_not_optimize & df_sexe_bio.Is_rev_max].SEXE.value_counts(normalize = True)
print df_sexe_bio[(~df_sexe_bio.Do_not_optimize) & df_sexe_bio.Is_rev_max].SEXE.value_counts(normalize = True)
print df_sexe_bio[df_sexe_bio.Do_not_optimize & ~df_sexe_bio.Is_rev_max].SEXE.value_counts(normalize = True)

1    0.64651
2    0.35349
dtype: float64
1    0.692142
2    0.307858
dtype: float64
2    0.646567
1    0.353433
dtype: float64


In [65]:
filter_1 = df_sexe_bio.Has_most_child & df_sexe_bio.Is_rev_max
filter_2 = df_sexe_bio.Do_not_optimize & df_sexe_bio.Has_most_child & ~df_sexe_bio.Is_rev_max
filter_3 = ~df_sexe_bio.Do_not_optimize & df_sexe_bio.Has_most_child & df_sexe_bio.Is_rev_max

In [66]:
print df_sexe_bio[filter_1 ].SEXE.value_counts(normalize = True)
print df_sexe_bio[filter_2].SEXE.value_counts(normalize = True)
print df_sexe_bio[filter_3].SEXE.value_counts(normalize = True)

1    0.670003
2    0.329997
dtype: float64
2    0.740288
1    0.259712
dtype: float64
1    0.689648
2    0.310352
dtype: float64


In [67]:
df_sexe_bio[df_sexe_bio.Has_most_child].SEXE.value_counts(normalize = True)

1    0.584007
2    0.415993
dtype: float64

In [68]:
print df_sexe_bio[filter_1 ].SEXE.value_counts(normalize = True) - df_sexe_bio[filter_3 ].SEXE.value_counts(normalize = True)


1   -0.019645
2    0.019645
dtype: float64


In [69]:
print 0.642497 - 0.772978 , 0.240302 - 0.362988

-0.130481 -0.122686


Diff in diff

In [70]:
print 67-64
print 67-76

3
-9


In [71]:
print 33-36
print 33-24

-3
9


In [72]:
import statsmodels.api as sm

In [73]:
#df_sexe.head()

### Pas biologiques

In [74]:
df_sexe["SEXE"] = (df_sexe.SEXE -1 ).astype('int')
df_sexe["Intersect"] = 1
Logit = sm.Logit(df_sexe["SEXE"], df_sexe[["Is_rev_max", "Has_most_child","Intersect" ]].astype('float')).fit()
Logit.summary() #Impact d'être une femme

Optimization terminated successfully.
         Current function value: 0.623530
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   SEXE   No. Observations:               120650
Model:                          Logit   Df Residuals:                   120647
Method:                           MLE   Df Model:                            2
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.1004
Time:                        11:04:13   Log-Likelihood:                -75229.
converged:                       True   LL-Null:                       -83628.
                                        LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Is_rev_max        -1.7618      0.015   -119.109      0.000        -1.791    -1.733
Has_most_child     0.6130      0.015     41.451      0.000         0.584     0.642
Intersect          0.5920      0.009     62.822      0.000         0.574     0.610
==================================================================================
"""

### BIologiques 

In [75]:
df_sexe_bio["SEXE"] = (df_sexe_bio.SEXE -1 ).astype('int')

In [76]:

df_sexe_bio["Intersect"] = 1
Logit = sm.Logit(df_sexe_bio["SEXE"], df_sexe_bio[["Is_rev_max", "Has_most_child","Intersect" ]].astype('float')).fit()
Logit.summary() #Impact d'être une femme

Optimization terminated successfully.
         Current function value: 0.622820
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   SEXE   No. Observations:                55694
Model:                          Logit   Df Residuals:                    55691
Method:                           MLE   Df Model:                            2
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.1015
Time:                        11:04:13   Log-Likelihood:                -34687.
converged:                       True   LL-Null:                       -38604.
                                        LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Is_rev_max        -1.6979      0.023    -74.518      0.000        -1.743    -1.653
Has_most_child     0.2702      0.023     11.858      0.000         0.226     0.315
Intersect          0.7218      0.014     52.543      0.000         0.695     0.749
==================================================================================
"""

In [77]:

df_sexe_bio["Intersect"] = 1
Logit = sm.Logit(df_sexe_bio["SEXE"], df_sexe_bio[["Is_rev_max", "Has_most_child","Intersect" ]].astype('float')).fit()
Logit.summary() #Impact d'être une femme

Optimization terminated successfully.
         Current function value: 0.622820
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                   SEXE   No. Observations:                55694
Model:                          Logit   Df Residuals:                    55691
Method:                           MLE   Df Model:                            2
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.1015
Time:                        11:04:13   Log-Likelihood:                -34687.
converged:                       True   LL-Null:                       -38604.
                                        LLR p-value:                     0.000
==================================================================================
                     coef    std err          z      P>|z|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
Is_rev_max        -1.6979      0.023    -74.518      0.000        -1.743    -1.653
Has_most_child     0.2702      0.023     11.858      0.000         0.226     0.315
Intersect          0.7218      0.014     52.543      0.000         0.695     0.749
==================================================================================
"""

In [78]:
df_sexe_bio.SEXE.value_counts()

1    27849
0    27845
dtype: int64

In [79]:

df_sexe_bio["Intersect"] = 1
Logit = sm.Logit(df_sexe_bio["Has_most_child"], df_sexe_bio[[ "Is_rev_max","SEXE","Intersect" ]].astype('float')).fit()
Logit.summary() #Impact d'être une femme

Optimization terminated successfully.
         Current function value: 0.523645
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         Has_most_child   No. Observations:                55694
Model:                          Logit   Df Residuals:                    55691
Method:                           MLE   Df Model:                            2
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.2445
Time:                        11:04:13   Log-Likelihood:                -29164.
converged:                       True   LL-Null:                       -38604.
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Is_rev_max     2.6578      0.023    116.640      0.000         2.613     2.702
SEXE           0.2702      0.023     11.858      0.000         0.226     0.315
Intersect     -1.4781      0.022    -67.317      0.000        -1.521    -1.435
==============================================================================
"""

In [80]:
np.exp(0.1520)

1.1641602364321124

In [81]:

df_sexe_bio["Intersect"] = 1
Logit = sm.Logit(df_sexe_bio["Has_most_child"], df_sexe_bio[[ "Is_rev_max","Intersect" ]].astype('float')).fit()
Logit.summary() #Impact d'être une femme

Optimization terminated successfully.
         Current function value: 0.524938
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         Has_most_child   No. Observations:                55694
Model:                          Logit   Df Residuals:                    55692
Method:                           MLE   Df Model:                            1
Date:                Wed, 10 Apr 2019   Pseudo R-squ.:                  0.2427
Time:                        11:04:13   Log-Likelihood:                -29236.
converged:                       True   LL-Null:                       -38604.
                                        LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Is_rev_max     2.5496      0.021    124.295      0.000         2.509     2.590
Intersect     -1.2885      0.015    -88.126      0.000        -1.317    -1.260
==============================================================================
"""

In [82]:
np.exp(2.5627)

12.970791215813765

In [83]:
1/np.exp(2.5627)

0.077096299166454649

In [84]:
np.exp(2.5627)

12.970791215813765

In [85]:
import datetime
print datetime.datetime.now()

2019-04-10 11:04:13.661000
